# <font color = 'orangered' size=6>Libraries import</font>   

In [1]:
# Import libraries
import os
import subprocess
import shutil
import re
import scipy
import numpy as np
import itertools
import pandas as pd
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from statannot import add_stat_annotation
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from math import pi

# <font color = 'orangered' size=6>Image preprocessing </font>   

# <font color = '#32CD32'>0. Nifti dimentions converter </font>

><font size='5'> 1. Uploads file from '**folder_path**' </font>
>
><font size='5'> 2. Please type your voxel size (um) after cell initialization one by one in (X, Y, Z) order  </font>
>
><font size='5'> 3. Returns converted files to '**out_path**' folder </font>


The '**matlab_script_converter_nifti**', '**folder_path**' and '**out_path**' need to be defined

In [ ]:
matlab_script_path = r'C:\Folder_with_images\convertNIfTI.m'  # Define path to the MATLAB script

folder_path = r'C:\Folder_with_images'  # Define the path to the folder containing the images

out_path = rf'{folder_path}\Scaled'  # Define the path to an output folder

messages = ['Input X (um): ', 'Input Y (um): ', 'Input Z (um): ']   # Voxel dimentions settings
dim1, dim2, dim3 = [float(input(msg)) for msg in messages]
matlab_script_dir = os.path.dirname(matlab_script_path)
matlab_command = f'matlab -nodesktop -nosplash -r "cd(\'{matlab_script_dir}\'); convertNIfTI(\'{folder_path}\', \'{out_path}\', {dim1}, {dim2}, {dim3}); exit;"'
subprocess.run(matlab_command, shell=True)


# <font color = '#32CD32'>1.  Paths and tags</font>


><font size='5'>1. Define the path to the folder with images after scaling
>  
><font size='5'>2. Select the files of interest by specifying the 'tag'

In [ ]:
folder_path = input("Enter the folder path: ")
tag = input("Enter the tag for file sorting: ")

print(f'Introduced path:                      {folder_path}')
print(f'Introduced tag for file selection:    {tag}')

# Sort files and copy files into a new folder

In [ ]:
def copy_and_rename_files(folder_path, tag):

    """
        Function creates a new folder with the name specified by the tag parameter within the given folder_path.
        A list of files is retrieved and sorted alphabetically in the folder_path.
        The function iterates over each file in the sorted list and extracts the file name without the extension.
        A new file name is constructed using the extracted file name and the provided tag.
        If the tag is detected in the file name, the function proceeds to the subsequent steps.
          Otherwise, a message is printed, indicating that the file name does not contain the matching tag.
        A new folder and a file path are created by combining the tag and the previous file path.
        The file is copied to the new file path using shutil.copy2.
        If the file is being used by another process, the copying operation is repeated up to max_retries times with a delay of 0.1 seconds between repeats.
        If the copying and renaming is successful, a message is printed, indicating the original file name and the new file name in the new folder.
    """

    tagname_folder = os.path.join(folder_path, f'{tag}')
    if not os.path.exists(tagname_folder):
        os.makedirs(tagname_folder)
    files = os.listdir(folder_path)
    files.sort()
    for i, file_name in enumerate(files):

        if tag in file_name:
            file_name_for_strings = file_name.split(".")[0]
            new_file_name = f"{file_name_for_strings}"
            file_extension = os.path.splitext(file_name)[1]
            new_file_path = os.path.join(tagname_folder, new_file_name + file_extension)
            max_retries = 3
            current_retry = 0
            while current_retry < max_retries:
                    try:
                        shutil.copy2(os.path.join(folder_path, file_name), new_file_path)
                        print(f"Copied '{file_name}' as '{new_file_name}{file_extension}' to new folder.")
                        break
                    except PermissionError:
                        current_retry += 1
        else:
            print(f'{file_name}  The filename does not match the pattern')

copy_and_rename_files(folder_path, tag)             # Call the function to copy and rename the files

# <font color = '#32CD32'>2. Image preparation</font>

# Provide the path and tag of files to the Avizo 'Main Python Console'

><font size='5'>Run the cell and paste the following **text output** into Avizo 'Main Python Console'</font>

In [ ]:
new_path = f'{folder_path}\{tag}'

print(f"new_path = r'{new_path}'")
print(f"tag = '{tag}'")

# <font>ROI selection (this step is optional, if not needed, you can skip it and move to Image Processing)</font>

><font size='5'>Paste the following **code** into Avizo console</font>
>
><font size='5'>Now you will work in the Avizo project window: </font>  
>
><font size='5'>Your images are uploaded</font>  
>
><font size='5'>ROI selection module 'Volume Edit' is launched</font>  
>
><font size='5'>'Volume Rendering' is created (but inactive) </font>  
>
><font size='5'>Continue with manual adjustments and ROI selection</font>  

In [ ]:
import os

files_0 = os.listdir(new_path)
files = []

volume_edit_counter = 1            # Number of volume edits

extention = '.nii'             # File extention variable
                               # Change it if you want to open another file types

for file in files_0:

    if file.endswith(extention):
        file_path = os.path.join(new_path, file)
        some_obj = hx_project.load(file_path)
        file = file.replace(" ", "-")
        if volume_edit_counter > 1:
            files.append(file.replace(".nii", f"({volume_edit_counter}).mask"))
        else:
            files.append(file.replace(".nii", ".mask"))
        # Volume rendering
        volumeRenderingSettings = hx_object_factory.create('HxVolumeRenderingSettings')
        volumeRender = hx_object_factory.create('HxVolumeRender2')
        hx_project.add(volumeRenderingSettings)
        hx_project.add(volumeRender)
        volumeRender.ports.volumeRenderingSettings.connect(volumeRenderingSettings)
        volumeRenderingSettings.ports.data.connect(hx_project.get(file))
        # 3D edges setting in Volume rendering
        volumeRenderingSettings.ports.effects.toggles[0].checked = HxPortToggleList.Toggle.CHECKED
        volumeRenderingSettings.viewer_mask = False
        volumeRenderingSettings.update()
        volumeRender.update()
        volumeRender.ports.colormap.auto_adjust_range = False
        volumeRenderingSettings.ports.data.connect(hx_project.get(file))
        volumeRender.update()

        for iter in range(volume_edit_counter):
            volume_edit= hx_object_factory.create('HxVolumeEdit')
            hx_project.add(volume_edit)
            print(iter)

            if iter == 0:
                volume_edit.ports.data.connect(hx_project.get(file))
                volume_edit.execute()
                file_from_volren_first = file.replace(".nii", ".modif")
                print(file_from_volren_first)

            else:
                volume_edit.ports.data.connect(hx_project.get(file_from_volren_first))
                volume_edit.execute()
                file_from_volren_first = file.replace(".nii", f"({iter+1}).modif")
                print(file_from_volren_first)


# Manual selection of ROI 
><font size='5'>1. Select ROI of your choice using **'Cut'** 'Inside/Outside'</font>  
>  
><font size='5'>2. **'Create mask'** using the 'Volume Edit' tool in Avizo</font>

# Saving

><font size='5'>Paste the following **code** into Avizo console</font>

In [ ]:
for file in files:
    # Get volume fraction
    get_volume_fraction = hx_object_factory.create('label_ratio')
    hx_project.add(get_volume_fraction)
    get_volume_fraction.ports.inputImage.connect(hx_project.get(file))
    get_volume_fraction.execute()
    file_get_volume_fraction = file.replace('.thresholded', '.measure')

hx_project.save(os.path.join(new_path, f'{tag}_ROI_autosave.hx'))

><font size='5'>Clear 'Project view' in Avizo</font>
>  
><font size='5'>Clear and restart the Python console</font>
>  
><font size='5'>Copy the **new_path** and **tag** into the Avizo console again (see beginning of Step 2)</font>

## <font color = 'orangered' size=6>Image processing </font>

# <font color = '#32CD32'>3. Process the image and save the results </font>
 Warning: this step requires computer RAM resources.  
 We do not recommend to upload stacks of large images.

><font size='5'>For images with extentions other than .nii:</font>   
><font size='4'>Change the **'extention'** variable</font>   

><font size='5'>Paste the following **code** into Avizo console</font>  

><font size='5'>Filter settings can be adjusted with respect to your image properties </font>  
><font size='4'>See the settings below</font>  

In [ ]:
import os

files = []
all_ranges = []
list_for_newData =[]
files_from_unsharpMask = []
ranges_for_data = []
files_from_threshold = []
files_from_spatial_graph_statistics = []

#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
extention = '.modif'            # File extention variable
                                # Can be set for various file types: .nii, .modif, .tiff, etc.

number_of_images_for_threshold = 3    # Set number of images in the series
                                      # (if number_of_images_for_threshold = 0, the threshold will be calculated for each image)

first_point_shift = 0.985    # The percentage by which the left autorange limit is moved
                             # Calculated as (default_autorange - default_autorange * first_point_shift)

second_point_shift = 0.9     # The percentage by which the right autorange limit is moved
                             # Calculated as (default_autorange - default_autorange * second_point_shift)

multiplier_for_frameshift_of_volren = 1.2  # Range frameshift for the Volume Rendering filter
                                         # If the image is too bright, adjust the settings respectively

threshold_multiplier = 2.8    # Threshold shift multiplier to avoid noise and mess
                              # Removes ranges from ranges frame that a visualized by threshold_multiplier
                              # Calculated as:
                              # ((first_point_shift/second_point_shift * ranges from autothreshold) * multiplier_for_frameshift_of_volren)) * threshold_multiplier
#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

if extention == '.modif':
    new_path = f'{new_path}\{tag}_ROI_autosave-files'
    files_0 = os.listdir(new_path)
else:
    files_0 = os.listdir(new_path)

    # Image upload and title modification
for file in files_0:
    if file.endswith(extention):
        print(file)
        file_path = os.path.join(new_path, file)
        some_obj = hx_project.load(file_path)
        file = file.replace(" ", "-")
        files.append(file)

            # Hessian filter
        hessian_filter = hx_object_factory.create('structureenhancementfilter')
        hx_project.add(hessian_filter)
        hessian_filter.ports.inputImage.connect(hx_project.get(file))

        #//////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        hessian_filter.ports.interpretation.selected = 1                      # Hessian filter: 3D (or XY) interpretation mode settings
        hessian_filter.ports.standardDeviationMinMax.texts[0].value = 0.8     # Hessian filter: standard deviation MIN
        hessian_filter.ports.standardDeviationMinMax.texts[1].value = 2.5      # Hessian filter: standard deviation MAX
        hessian_filter.ports.standardDeviationStep.texts[0].value = 0.8       # Hessian filter: standard deviation STEP
        #//////////////////////////////////////////////////////////////////////////////////////////////////////////////////

            # Hessian filter: type settings
        hessian_filter.ports.structureType.menus[0] = HxPortButtonMenu.Menu(options = ['Rod', 'Ball', 'Plane'], selected = 0)
        hessian_filter.update()
        hessian_filter.execute()
        hessian_filter.fire()
        file_from_hessian_filter = file.replace(extention, ".filtered")

            # Unsharp masking filter
        unsharpMask = hx_object_factory.create('unsharpmaskfilter')
        hx_project.add(unsharpMask)

        #//////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        unsharpMask.ports.interpretation.selected = 0         # Unsharp mask: interpolation type
        unsharpMask.ports.edgeSize.texts[0].value = 5         # Unsharp mask: edge size settings
        unsharpMask.ports.edgeContrast.texts[0].value = 0.99  # Unsharp mask: edge contrast settings
        #//////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        
        unsharpMask.ports.inputImage.connect(hx_project.get(file_from_hessian_filter))
        unsharpMask.execute()
        unsharpMask.fire()
        if file.endswith(extention):
            file_from_unsharpMask = file.replace(extention, "(2).filtered")
        else:
            assert 'Please rename your file'
        files_from_unsharpMask.append(file_from_unsharpMask)
    else:
        continue

# Volume rendering
volumeRenderingSettings = hx_object_factory.create('HxVolumeRenderingSettings')
volumeRender = hx_object_factory.create('HxVolumeRender2')
hx_project.add(volumeRenderingSettings)
hx_project.add(volumeRender)
volumeRender.ports.volumeRenderingSettings.connect(volumeRenderingSettings)
volumeRenderingSettings.ports.data.connect(hx_project.get(file_from_unsharpMask))
# Volume rendering: 3D edge settings
volumeRenderingSettings.ports.effects.toggles[0].checked = HxPortToggleList.Toggle.CHECKED

# Autorange collection
for file in files_from_unsharpMask:
    ranges = []
    volumeRenderingSettings.fire()
    volumeRender.ports.colormap.auto_adjust_range = True
    volumeRenderingSettings.ports.data.connect(hx_project.get(file))
    volumeRender.update()

    if number_of_images_for_threshold > 0:
        if 'thr' in file:
            range_for_spec = volumeRender.ports.colormap.range
            # Autorange recalculation
            ranges.append(range_for_spec[0] - range_for_spec[0] * first_point_shift)
            ranges.append(range_for_spec[1] - range_for_spec[1] * second_point_shift)
            for iter in range(number_of_images_for_threshold):
                all_ranges.append(ranges)

    elif number_of_images_for_threshold == 0:
        range_for_spec = volumeRender.ports.colormap.range
        # Autorange recalculation
        if range_for_spec[0] - range_for_spec[0] * first_point_shift == 0:
            ranges.append((range_for_spec[1] - range_for_spec[1] * first_point_shift)/10)
        else:
            ranges.append(range_for_spec[0] - range_for_spec[0] * first_point_shift)
        ranges.append(range_for_spec[1] - range_for_spec[1] * second_point_shift)
        all_ranges.append(ranges)

# Prints the list of shifted autoranges
print('\n All shifted ranges: \n ___________________________________ \n', all_ranges)

_ = 0
# Volume Rendering: jumping settings
for file in files_from_unsharpMask:
    volumeRenderingSettings.fire()
    volumeRender.ports.colormap.auto_adjust_range = False
    volumeRender.update()
    volumeRenderingSettings.ports.data.connect(hx_project.get(file))

    # Range percentage multiplier
    range_for_volren = [element * multiplier_for_frameshift_of_volren for element in all_ranges[_]]
    ranges_for_data.append(range_for_volren)
    print(f'\n Range for correct volume rendering [{_}]: \n ___________________________________ \n' , range_for_volren)
    volumeRender.ports.colormap.range = range_for_volren
    volumeRender.ports.alphaScale.value = 0.4              # Transparency settings
    volumeRenderingSettings.ports.rendering.selected = 0   # Standard type rendering settings
    volumeRenderingSettings.fire()
    _+=1

_ = 0
# Interactive threshold
for file_from_unsharpMask in files_from_unsharpMask:

    # Interactive threshold: recalculation
    range_for_threshold =  [element * threshold_multiplier for element in ranges_for_data[_]]
    print(f'\n Range for thresholding [{_}]: \n ___________________________________ \n', range_for_threshold)
    interactive_threshold = hx_object_factory.create('HxInteractiveThreshold')
    hx_project.add(interactive_threshold)
    interactive_threshold.ports.data.connect(hx_project.get(file_from_unsharpMask))

    # Interactive threshold: 3D preview type settings
    interactive_threshold.ports.preview.toggles[0].checked = HxPortToggleList.Toggle.UNCHECKED
    interactive_threshold.ports.preview.toggles[1].checked = HxPortToggleList.Toggle.CHECKED
    interactive_threshold.update()
    interactive_threshold.ports.intensityRange.range = range_for_threshold
    interactive_threshold.update()
    interactive_threshold.apply_transform_to_result = False
    interactive_threshold.execute()
    interactive_threshold.fire()
    _+=1
    file_from_threshold = file_from_unsharpMask.replace(".filtered", ".thresholded")
    files_from_threshold.append(file_from_threshold)
    interactive_threshold.viewer_mask = False
    interactive_threshold.update()
    test_export = hx_project.get(file_from_threshold)

for file in files_from_threshold:

    # Pruning: filtering for thresholded binary images
    pruning= hx_object_factory.create('pruning3d')
    hx_project.add(pruning)
    pruning.ports.inputBinaryImage.connect(hx_project.get(file))
    pruning.ports.numberOfIterations.texts[0].value = 10        # Pruning: number of iterations
    pruning.execute()
    pruning.fire()
    file_from_pruning = file.replace('.thresholded', '.pruned')

    # Collection of volume fraction
    get_volume_fraction = hx_object_factory.create('label_ratio')
    hx_project.add(get_volume_fraction)
    get_volume_fraction.ports.inputImage.connect(hx_project.get(file_from_pruning))
    get_volume_fraction.execute()
    file_get_volume_fraction = file.replace('.thresholded', '.measure')

    # Centerline tree (for thresholded binary images)
    centrline_tree = hx_object_factory.create('HxTEASAR')
    hx_project.add(centrline_tree)
    centrline_tree.ports.data.connect(hx_project.get(file_from_pruning))
    centrline_tree.ports.tubesParams.texts[0].value = 2        # Centrline tree Slope
    centrline_tree.ports.tubesParams.texts[1].value = 4        # Centrline tree ZeroVal
    centrline_tree.update()
    centrline_tree.execute()
    centrline_tree.fire()
    file_centrline_tree = file.replace('.thresholded', '.Spatial-Graph')

    # Collecting statistics from Centerline tree using Spartial Graph Statistics
    spatial_graph_statistics = hx_object_factory.create('HxSpatialGraphStats')
    hx_project.add(spatial_graph_statistics)
    spatial_graph_statistics.ports.data.connect(hx_project.get(file_centrline_tree))
    # Attribute graphs and Statistics: Toggle settings
    spatial_graph_statistics.ports.output.toggles[0].checked = HxPortToggleList.Toggle.CHECKED
    spatial_graph_statistics.ports.output.toggles[1].checked = HxPortToggleList.Toggle.CHECKED
    spatial_graph_statistics.execute()
    spatial_graph_statistics.fire()
    file_from_spatial_graph_statistics = file.replace('.thresholded', '.attributegraph')
    files_from_spatial_graph_statistics.append(file_from_spatial_graph_statistics)

# Autosaving your project in Avizo
if extention == '.modif':
    hx_project.save(os.path.join(os.path.dirname(new_path), f'{tag}_autosave.hx'))

else:
    hx_project.save(os.path.join(new_path, f'{tag}_autosave.hx'))

# Export of Attributegraphs from Spatial Graph Statistics as .XML files
for file_from_spatial_graph_statistics in files_from_spatial_graph_statistics:

    out = hx_project.get(file_from_spatial_graph_statistics)
    out.update()
    out.update()
    out.ports.show.buttons[1].hit = True
    out.update()
    out.ports.show.buttons[1].hit = False
    out.update()
    out.update()

><font size='5'>Press 'Autosave' in Avizo to save project  </font>
>
><font size='5'>Save .XML files to the folder with images</font>

# <font color = 'orangered' size=6>Output Data processing </font>   

# <font color = '#32CD32'>4. Script for reading and filtering (denoising) the .XML output from Avizo</font>
><font size = 5>Set 'file_path' to a folder with .XML data  </font>  
>
><font size = 5>Define the 'pattern' variable to get an appropriate title for 'Condition' labeling </font>  
>
><font size = 5>Define the 'selector' and select the column name to apply a threshold to remove the noise from the data.</font>
>
><font size = 5>Define 'threshold' variable for better results  </font>

In [2]:
file_path = rf'C:\Folder_with_images\Scaled\Mouse'   # Path to the folder with preprocessed images

tag = "Mouse"

is_ROI_selected = 'Yes'                 # (Yes/No) If we are using ROI in our images

pattern = r'^([a-zA-Z0-9_-]+).*'       # Select pattern for 'Condition' titling

selector = 'ChordLength'               # Select column name to apply the threshold for noise filtering
                                       # Use ChordLength' or 'CurvedLength' for the best result
threshold =  40000                     # Drop all the values outside the threshold

><font size = 5>Run the functions below</font>

In [3]:
def extract_values_from_file(autosaved_directory_path):

    ''' Extraction and converting of volume values and ratios to dataframe format '''

    name = ''
    value_2 = None
    value_3 = None
    value_4 = None
    value_5 = None
    value_6 = None

    # Define regular expressions to match the desired patterns
    name_pattern = r'Filename "(.*?)"'
    value_pattern = r'@(\d+)\s+([\d.e+-]+)'

    # Open and read the file
    with open(autosaved_directory_path, 'r') as file:
        content = file.read()

        # Extract the name
        name_match = re.search(name_pattern, content)
        if name_match:
            name = name_match.group(1)

        # Extract values using regular expressions
        value_matches = re.finditer(value_pattern, content)
        for match in value_matches:
            key = match.group(1)
            val = match.group(2)
            if key == '2':
                value_2 = float(val)
            elif key == '3':
                value_3 = float(val)
            elif key == '4':
                value_4 = float(val)
            elif key == '5':
                value_5 = float(val)
            elif key == '6':
                value_6 = float(val)

    return {
        'Filename': os.path.basename(name),
        'Volume Fraction': value_2,
        'Label Volume': value_3,
        'Total Volume': value_4,
        'Label Voxel Count': value_5,
        'Total Voxel Count': value_6
    }

def get_segments_sheet(file):
    """
        A function to process .XML file, and then create and return a pandas DataFrame.
        It loads the XML file and loads the 'Segments' sheet from the three output sheets of an .xml file from Avizo.
        The function is iterated over the rows in the sheet and over the cells in the row to get the value of each cell.
        Return pandas DataFrame.
    """
    tree = ET.parse(file)
    root = tree.getroot()
    segments_sheet = root.find(".//ss:Worksheet[@ss:Name='Segments']", namespace_map)

    data_rows = []
    for row in segments_sheet.findall('.//ss:Row', namespace_map):
        data = []
        for cell in row.findall('.//ss:Cell', namespace_map):
            data.append(cell.find('.//ss:Data', namespace_map).text)
        data_rows.append(data)

    df = pd.DataFrame(data_rows)
    df.columns = df.iloc[0]
    df = df[1:]

    return df

def get_nodes_sheet(file):
    """
        A function to process .XML file, and then create and return a pandas DataFrame.
        It loads the XML file and loads the 'Nodes' sheet from the three output sheets of an .xml file from Avizo.
        The function is iterated over the rows in the sheet and over the cells in the row to get the value of each cell.
        Return pandas DataFrame.
    """
    tree_nodes = ET.parse(file)
    root_nodes = tree_nodes.getroot()
    segments_sheet_nodes = root_nodes.find(".//ss:Worksheet[@ss:Name='Nodes']", namespace_map)

    data_rows_nodes = []
    for row in segments_sheet_nodes.findall('.//ss:Row', namespace_map):
        data_nodes = []
        for cell in row.findall('.//ss:Cell', namespace_map):
            data_nodes.append(cell.find('.//ss:Data', namespace_map).text)
        data_rows_nodes.append(data_nodes)

    df_nodes = pd.DataFrame(data_rows_nodes)
    df_nodes.columns = df_nodes.iloc[0]
    df_nodes = df_nodes[1:]

    return df_nodes

def count_symbols(df):
    """
    Function to count the values
    """
    symbol_counts = df['Coordination Number'].value_counts()
    symbol_1_count = 0
    other_symbols_count = 0
    max_symbol_value = 0

    for symbol, count in symbol_counts.items():
        if int(symbol) == 1:
            symbol_1_count = count
        else:
            other_symbols_count += count

        if int(symbol) > int(max_symbol_value):
            max_symbol_value = symbol

    return symbol_1_count, other_symbols_count, max_symbol_value

In [ ]:
namespace_map = { 'ss': 'urn:schemas-microsoft-com:office:spreadsheet' } # Namescape parameter for reading the Avizo output.xml

data = []
ROI_data = []

if is_ROI_selected == 'Yes':
    ROI_directory_path = f'{file_path}\{tag}_ROI_autosave-files'
    Vessels_directory_path = f'{file_path}\{tag}_autosave-files'
    print('We work with ROI')
elif is_ROI_selected == 'No':
    Vessels_directory_path = f'{file_path}\{tag}_autosave-files'
    ROI_directory_path = f'{file_path}\{tag}_autosave-files'
    print('We work without ROI')

if os.path.exists(ROI_directory_path) and os.path.isdir(ROI_directory_path):
    list_of_ROI = os.listdir(ROI_directory_path)
    if len(list_of_ROI) > 0:
        for filename in os.listdir(ROI_directory_path):
            if filename.endswith('.measure'):
                autosaved_ROI_directory_path = os.path.join(ROI_directory_path, filename)
                ROI_extracted_data = extract_values_from_file(autosaved_ROI_directory_path)
                ROI_data.append(ROI_extracted_data)
    else:
        print(ROI_directory_path)
        print("Warning: No files with ROI in folder.")
else:
    print(ROI_directory_path)
    print("Warning: folder with ROI does not exist.")

for filename in os.listdir(Vessels_directory_path):
    if filename.endswith('.measure'):
        autosaved_directory_path = os.path.join(Vessels_directory_path, filename)
        extracted_data = extract_values_from_file(autosaved_directory_path)
        data.append(extracted_data)

Vessels_volume_relations = pd.DataFrame(data)
ROI_volume_relations = pd.DataFrame(ROI_data)

dataframes = []                     # List to store DataFrames for each file
dataframes_names = []               # List to store DataFrames name for each file
processed_dataframes = []           # List to store processed DataFrames

for file in os.listdir(file_path):
    if file.endswith(".xml"):                               # Ensures that the file is an .XML file
        full_path = os.path.join(file_path, file)           # Shows the full file path
        df = get_segments_sheet(full_path)                  # Processes the .XML file and creates a DataFrame
        df_nodes = get_nodes_sheet(full_path)

        # Assuming you have loaded the dataset into a pandas DataFrame called df_nodes
        symbol_1_count, other_symbols_count, max_symbol_value = count_symbols(df_nodes)
        match = re.match(pattern, file)

        df['Number of empty endpoints'] = symbol_1_count
        df['Number of branchpoints'] = other_symbols_count
        df['Max Coordination Number'] = max_symbol_value

        if match:
            extracted_text = match.group(1)
        df = df.rename(columns={'Segment ID': f'{extracted_text}'})
        dataframes.append(df)
        dataframes_names.append(file)

        # Specification of the threshold for DataFrames
        # Filtering of the DataFrames in accordance with the threshold variable
i=0
for df in dataframes:

    if selector in df.columns:                          # Ensures that the 'selector' variable exists in columns
        df[selector] = pd.to_numeric(df[selector])      # Converts 'ChordLength' column to numeric
        mean_length = df[selector].mean()               # Calculates the mean of column values
        dropped_count = len(df[df[selector] < threshold])

        df['Number of empty endpoints'] = df['Number of empty endpoints'] - dropped_count * 2
        df['Dropped segments'] = dropped_count
        df = df[df[selector] >= threshold]              # To refresh the dataset

        if is_ROI_selected == 'Yes':
            vessels_fraction =  Vessels_volume_relations['Volume Fraction'][i]
            ROI_fraction = ROI_volume_relations['Volume Fraction'][i]
        elif is_ROI_selected == 'No':
            vessels_fraction =  Vessels_volume_relations['Volume Fraction'][i]
            ROI_fraction = Vessels_volume_relations['Volume Fraction'][i]
        print(
              f'______________________________________________',
              f'\n{dataframes_names[i]} - {i+1}',
              f'\nRelation of ROI to whole image = {round(ROI_fraction, 6)}          Ratio of Vessels Mask to background = {round(vessels_fraction, 6)}',
              f'\nMean of {selector} = {round(mean_length, 2)};',
              f'   Threshold of {selector} = {threshold};       Dropped = {dropped_count} samples',
              )
        processed_dataframes.append(df)

    else:
        print('Column with provided name does not exist')

    for column in processed_dataframes[i].columns:
        if processed_dataframes[i][column].dtype == 'object':
            try:
                processed_dataframes[i][column] = processed_dataframes[i][column].astype(float)
            except ValueError:
                if column == 'Point IDs':
                    print()
                else:
                    print(f"Column in {dataframes_names[i]} with name '{column}' couldn't be converted to float64")
    i+=1

# <font color = '#32CD32'>5. Creation of datasets with a defined order</font>

><font size = 5>Define the dataset order for merging     </font>  
(e.g. if there are 4 datasets that need to be splitted into two groups (based on filename), specify 2, 2 in **merging_order**

><font size = 5>Merging allows to create dataframes that are specifically suited for your sampling conditions   </font>
    
><font size = 5>One of the outputs is a merged DataFrame that contains a column named 'conditions' with the filenames of all the images that were analyzed </font>

><font size = 5>The other one is 'mean' dataset according to the 'merging order' and mode </font>  
>This dataset we save in .xlsx format, that are easy to analyse

In [5]:
# Define the merging order
merging_order = [1]*3
# Save frame of Mean values according to path
output_path_for_frame_saving = f'{file_path}\{tag}.xlsx'
pd.options.display.max_columns = 50

In [6]:
dataframes = processed_dataframes
merged_dataframes = []
start_index = 0
for group_size in merging_order:
    end_index = start_index + group_size
    merged_df = pd.concat(dataframes[start_index:end_index])
    merged_dataframes.append(merged_df)
    start_index = end_index
for i, merged_df in enumerate(merged_dataframes):
    merged_df = merged_df.reset_index(drop=True)
    merged_dataframes[i] = merged_df

In [7]:
one_big_merged_frame = pd.DataFrame()
frame_for_export = pd.DataFrame()
labels = []
mean_frames = []

for dataset in merged_dataframes:
    condition_name = dataset.columns[0]
    dataset['Condition'] = condition_name
    labels.append(condition_name)

    dataset['CurvedLength'] = dataset['CurvedLength'] / 1000    # in [um]
    dataset['ChordLength'] = dataset['ChordLength'] / 1000      # in [um]
    dataset['MeanRadius'] = dataset['MeanRadius'] / 1000        # in [um]
    dataset['Volume'] = dataset['Volume'] / 10**9               # in [um^3]

    dataset['Volume_Sum_perImage [um^3]'] = dataset['Volume'].sum()

    dataset['Geometrical_Volume_perImage [um^3]'] = np.sum((dataset['MeanRadius'])**2 * pi * dataset['CurvedLength']) 

    dataset['Imaging_Error_perImage'] = 1 - dataset['Geometrical_Volume_perImage [um^3]'] / dataset['Volume_Sum_perImage [um^3]']

    curvedlength_sum = dataset['CurvedLength'].sum()

    dataset['Vessels_CurvedLength_Sum_perImage [um]'] = curvedlength_sum 

    dataset['Weighted_MeanRadius_perImage [um]'] = np.sum(dataset['MeanRadius'] * dataset['CurvedLength']) / (curvedlength_sum)

    dataset['Weighted_MeanTortuosity_perImage'] = np.sum(dataset['Tortuosity'] * dataset['CurvedLength']) / (curvedlength_sum)

    dataset['Weighted_Segment_MeanRadius_perSegment'] = (dataset['MeanRadius'] * dataset['CurvedLength']) / (curvedlength_sum)

    dataset['Weighted_Segment_Volume_perSegment'] = (dataset['Volume'] * dataset['CurvedLength']) / (curvedlength_sum)
    
    selected_columns = (
                        dataset.columns[1:22].tolist() + ['Condition'] + ['Volume_Sum_perImage [um^3]'] +
                        ['Geometrical_Volume_perImage [um^3]'] + ['Imaging_Error_perImage'] +
                        ['Vessels_CurvedLength_Sum_perImage [um]'] + ['Weighted_MeanRadius_perImage [um]'] +
                        ['Weighted_MeanTortuosity_perImage'] + ['Weighted_Segment_MeanRadius_perSegment'] 
                        + ['Weighted_Segment_Volume_perSegment'] 
                        )

    dataset = dataset[selected_columns].copy()
    one_big_merged_frame = pd.concat([one_big_merged_frame, dataset], ignore_index=True)
    dataset.loc[:, 'Weighted_Segment_MeanRadius_perSegment'] = sum(dataset['Weighted_Segment_MeanRadius_perSegment'])
    dataset.loc[:, 'Weighted_Segment_Volume_perSegment'] = sum(dataset['Weighted_Segment_Volume_perSegment'])
    means = dataset.mean(skipna=True, numeric_only=True)
    mean_df = pd.DataFrame(means, columns=[f'{condition_name}'])
    
    mean_df = mean_df.T
    mean_frames.append(mean_df)

frame_for_export = pd.concat(mean_frames, axis=0)
frame_for_export['Filename'] = frame_for_export.index
frame_for_export.reset_index(drop=True, inplace=True)
frame_for_export.insert(0, 'Filename', frame_for_export.pop('Filename'))
frame_for_export['Total Image Volume [um^3]'] = Vessels_volume_relations['Total Volume'] 
frame_for_export['ROI Volume [um^3]'] = ROI_volume_relations['Label Volume'] 
frame_for_export['Vessels in ROI Volume (Labeled Volume after pruning before denoizing) [um^3]'] = Vessels_volume_relations['Label Volume'] 
frame_for_export['Ratio of ROI Mask to whole image'] = ROI_volume_relations['Volume Fraction'] 
frame_for_export['Ratio of Vessel Mask to whole image'] = Vessels_volume_relations['Volume Fraction'] 
division_result = frame_for_export['Vessels in ROI Volume (Labeled Volume after pruning before denoizing) [um^3]'] / frame_for_export['ROI Volume [um^3]']
frame_for_export['Ratio of Vessel Mask to background ROI'] = division_result
columns_to_drop = [col for col in frame_for_export.columns if col.startswith('Tensor') or col.startswith('Node') or col.startswith('Subgraph')]
frame_for_export.drop(columns=columns_to_drop, inplace=True)
writer = pd.ExcelWriter(output_path_for_frame_saving, engine='xlsxwriter')
frame_for_export.to_excel(writer, sheet_name='Sheet1', index=True)
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Set the width of Column 1
worksheet.set_column('A:A', 4)
worksheet.set_column('B:S', 13)
writer.close()

<font size='5' color = '#0E4C92'>Output dataframes:</font>

><font size='5'>frame_for_export</font>

Consists of 'mean' by dataframe observation.  
Each observation defined by sample group was provided in merging.
  
><font size='5'>one_big_merged_frame</font>

Consists of all dataframes with column 'Condition', where the sample names are presented

# <font color = 'orangered' size=6>Output representation </font>   

# <font color = '#32CD32'>6. Boxplots for all samples with respective stats and annotations</font>



><font size='5'>Here we work with 'one_big_merged_frame' dataset

Makes plots according to 'Condition' column and our groups

In [ ]:
col_name = 'CurvedLength'             # col_name = 'column_name'
                                    # Fill it with the name of the column you want to analyse

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
fig, axes = plt.subplots(figsize=(6, 6), dpi=100)
sns.boxplot(data=one_big_merged_frame, x='Condition', y=col_name, hue='Condition', palette='pastel', ax=axes, showfliers=0, dodge=False)
axes.set_xticklabels(axes.get_xticklabels(), rotation = -10)
legend = plt.legend( loc='upper left', bbox_to_anchor=(0.9, 1))
plt.title(col_name, pad = 20)
plt.show()

# SETS PER SEGMENT  
  
><font size='5'>Choose the feature set and run the selected cell</font>

SET 1

In [9]:
col_names_to_analyze_perSegment = ['Tortuosity', 
                                   'MeanRadius',
                                   'ChordLength', 
                                   'CurvedLength']

SET 2

In [10]:
col_names_to_analyze_perSegment = ['Volume', 
                                   'Weighted_Segment_MeanRadius_perSegment',
                                   'Weighted_Segment_Volume_perSegment']

><font size='5'>If you prefer the representation of your data in log scale - run the cell below</font>  
  
><font size='5'>Otherwise skip it</font>

In [11]:
for col in col_names_to_analyze_perSegment:
    one_big_merged_frame[col] = np.log10(one_big_merged_frame[col])

# perSegment features plotting (of the preselected 'SET #')

In [ ]:
num_subplots = len(col_names_to_analyze_perSegment)
fig, axes = plt.subplots(1, num_subplots, figsize=(num_subplots * 5, 7), dpi=750)
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
legend_handles = []
for i, col_name in enumerate(col_names_to_analyze_perSegment):
    ax = axes[i]
    sns.violinplot(data=one_big_merged_frame, x='Condition', y=col_name, hue='Condition', palette='pastel', ax=ax, showfliers=1, dodge=False)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=-90)
    ax.set_title(col_name, pad=30)
    ax.get_legend().remove()
    ax.set_xlabel('')

    if i == len(col_names_to_analyze_perSegment) - 1:
        handles, labels = ax.get_legend_handles_labels()
        legend_handles.extend(handles)
legend = plt.legend(legend_handles, labels, loc='upper left', bbox_to_anchor=(1, 0.6))

plt.tight_layout()
plt.show()

# SETS PER IMAGE  
  
><font size='5'>Choose the feature set and run the selected cell</font>

SET 1

In [13]:
col_names_to_analyze_perImage = ['Dropped segments',
                                 'Number of empty endpoints',
                                 'Number of branchpoints',
                    	         'Volume_Sum_perImage [um^3]']

SET 2

In [25]:
col_names_to_analyze_perImage = ['Geometrical_Volume_perImage [um^3]',
                                 'Imaging_Error_perImage',
                                 'Vessels_CurvedLength_Sum_perImage [um]',
                                 'Weighted_MeanRadius_perImage [um]',
                                 'Weighted_MeanTortuosity_perImage']

# perImage features plotting (of the preselected 'SET #')

In [ ]:
num_subplots = len(col_names_to_analyze_perImage)

fig, axes = plt.subplots(1, num_subplots, figsize=(num_subplots * 6, 6), dpi=750)
custom_params = {
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.labelsize": 16,  
    "axes.titlesize": 18,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "legend.fontsize": 16
}
sns.set_theme(style="ticks", rc=custom_params)

for i, col_name in enumerate(col_names_to_analyze_perImage):
    ax = axes[i]
    sns.boxplot(data=one_big_merged_frame, x='Condition', y=col_name, hue='Condition', palette='pastel', ax=ax, showfliers=1, dodge=False)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=-90)
    ax.set_title(col_name, pad=30)
    ax.get_legend().remove()
    ax.set_xlabel('')
    ax.set_ylabel('')

plt.subplots_adjust(wspace=0.3)
plt.show()

# <font color = '#32CD32'>7. Statistical tests</font>

><font size = 5>Choose above and run the cell with "SET #" of data (perImage or perSegment) you want to analyse</font>  



# Kruskall test

In [26]:
col_names_to_analyze = col_names_to_analyze_perImage

In [28]:
ks_statistic_total = []
ks_p_value_total = []
for col in col_names_to_analyze:
    print('Feature for test:', col)
    # Kolmogorov-Smirnov test
    conditions = one_big_merged_frame['Condition'].unique()
    box_pairs = list(itertools.combinations(conditions, 2))

Feature for test: Geometrical_Volume_perImage [um^3]
Feature for test: Imaging_Error_perImage
Feature for test: Vessels_CurvedLength_Sum_perImage [um]
Feature for test: Weighted_MeanRadius_perImage [um]
Feature for test: Weighted_MeanTortuosity_perImage


In [ ]:
def check_p_value_interval(p_value):
    p_value_legend = {
        'ns': (5.00e-02 < p_value <= 1.00e+00),
        '*': (1.00e-02 < p_value <= 5.00e-02),
        '**': (1.00e-03 < p_value <= 1.00e-02),
        '***': (1.00e-04 < p_value <= 1.00e-03),
        '****': (p_value <= 1.00e-04)
    }
    for key, interval_check in p_value_legend.items():
        if interval_check:
            return key
    return None

num_subplots = len(col_names_to_analyze)
fig, axes = plt.subplots(1, num_subplots, figsize=(num_subplots * 4, 7), dpi=750)
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
legend_handles = []

for i, col_name in enumerate(col_names_to_analyze):
    ax = axes[i]
    sns.boxplot(data=one_big_merged_frame, x='Condition', y=col_name, hue='Condition', palette='pastel', ax=ax, showfliers=1, dodge=False)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=-90)
    ax.set_title(col_name, pad=90)
    ks_number = len(one_big_merged_frame['Condition'].unique())
    sequence_of_data = list(range(1, ks_number+1))
    sequence_of_data_pairs = list(itertools.combinations(sequence_of_data, 2))

    ax.get_legend().remove()
    ax.set_xlabel('')
    if i == len(col_names_to_analyze) - 1:
        handles, labels = ax.get_legend_handles_labels()
        legend_handles.extend(handles)
    
    test_results = add_stat_annotation(ax, data=one_big_merged_frame, x='Condition', y=col_name, 
                                       box_pairs=box_pairs, test='Kruskal', text_format='star', 
                                       loc='outside', verbose=2)

legend = plt.legend(legend_handles, labels, loc='upper left', bbox_to_anchor=(1, 0.6))
plt.tight_layout()
plt.show()

# KS test

In [19]:
col_names_to_analyze = col_names_to_analyze_perSegment

In [ ]:
ks_statistic_total = []
ks_p_value_total = []
for col in col_names_to_analyze:
    print('Feature for test:', col)
    # Kolmogorov-Smirnov test
    conditions = one_big_merged_frame['Condition'].unique()
    box_pairs = list(itertools.combinations(conditions, 2))
    for pair in box_pairs:
        data1 = one_big_merged_frame[one_big_merged_frame.Condition == pair[0]][col]
        data2 = one_big_merged_frame[one_big_merged_frame.Condition == pair[1]][col]  
        ks_result = scipy.stats.ks_2samp(data1, data2)
        ks_statistic, p_value = scipy.stats.ks_2samp(data1, data2)
        ks_statistic_total.append(ks_statistic)
        ks_p_value_total.append(p_value)
        print(f'Samples pair:', pair, 'KS result:', ks_result)

><font size = 5>Type in the pairs for KS test</font>  

In [21]:
sequence_of_data_pairs = [(1,4), (2,5), (3,6)]

In [ ]:
def check_p_value_interval(p_value):
    p_value_legend = {
        'ns': (5.00e-02 < p_value <= 1.00e+00),
        '*': (1.00e-02 < p_value <= 5.00e-02),
        '**': (1.00e-03 < p_value <= 1.00e-02),
        '***': (1.00e-04 < p_value <= 1.00e-03),
        '****': (p_value <= 1.00e-04)
    }
    for key, interval_check in p_value_legend.items():
        if interval_check:
            return key
    return None

num_subplots = len(col_names_to_analyze)
fig, axes = plt.subplots(1, num_subplots, figsize=(num_subplots * 5, 8), dpi=750)
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
legend_handles = []

for i, col_name in enumerate(col_names_to_analyze):
    ax = axes[i]
    sns.violinplot(data=one_big_merged_frame, x='Condition', y=col_name, hue='Condition', palette='pastel', ax=ax, showfliers=0, dodge=False)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=-90, fontsize = 12)
    ax.set_title(col_name, pad=55)
    ks_number = 3
    sequence_of_data = list(range(1, ks_number+1))
    
    ks_annotations = [f'{sequence_of_data_pairs[k]}: {check_p_value_interval(ks_p_value_total[k+i*ks_number])}' for k in range(ks_number)]
    ax.annotate('\n'.join(ks_annotations),
            xy=(0.4, 1), xycoords='axes fraction',
            bbox=dict(boxstyle="round,pad=0.5", edgecolor='blue', facecolor='white'),
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0.3", color='blue'))
    ax.get_legend().remove()
    ax.set_xlabel('')
    if i == len(col_names_to_analyze) - 1:
        handles, labels = ax.get_legend_handles_labels()
        legend_handles.extend(handles)
legend = plt.legend(legend_handles, labels, loc='upper left', bbox_to_anchor=(1, 0.6))
plt.tight_layout()
plt.show()